In [1]:
from pytube import YouTube
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from preprocessing import Utilities

/anaconda/envs/openai/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
dest = '/home/elastic/notebooks/vector_search_applications/practice_data/impact_theory_video_ids.csv'

In [5]:
def create_video_url(video_id: str, playlist_id: str='PL8qcvQ7Byc3OJ02hbWJbHWePh4XEg3cvo'):
    return f'https://www.youtube.com/watch?v={video_id}&list={playlist_id}'

In [10]:
video_ids = pd.read_csv(dest).values.tolist()
video_ids = [v[0] for v in video_ids]

In [1]:
links = [create_video_url(vidid) for vidid in video_ids]
links[0]

NameError: name 'video_ids' is not defined

In [47]:
for link in links:
    try:
        video = YouTube(link)
        # print(f'Streams: {video.streams}')
        audio = video.streams.filter(only_audio=True, file_extension='mp4').first()
        audio.download(dest, skip_existing=False)
    except Exception as e:
        print(e)

In [60]:
yt_vids = []
for link in links:
    yt_vids.append(YouTube(link))

In [14]:
url = 'https://www.youtube.com/playlist?list=PLAiITlQWhsRTcIw_C4SHr0y_WqogzR8lR'

In [15]:
page = requests.get(url)

In [22]:
soup = BeautifulSoup(page.text, "html")

In [24]:
print(soup.prettify())

<!DOCTYPE html>
<html darker-dark-theme="" darker-dark-theme-deprecate="" lang="en" style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" system-icons="" typography="" typography-spacing="">
 <head>
  <script data-id="_gd" nonce="cbGzRXMBKOar9uUBXAdjWg">
   window.WIZ_global_data = {"MuJWjd":false,"nQyAE":{},"oxN3nb":{"1":false}};
  </script>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="ApvK67ociHgr2egd6c2ZjrfPuRs8BHcvSggogIOPQNH7GJ3cVlyJ1NOq/COCdj0+zxskqHt9HgLLETc8qqD+vwsAAABteyJvcmlnaW4iOiJodHRwczovL3lvdXR1YmUuY29tOjQ0MyIsImZlYXR1cmUiOiJQcml2YWN5U2FuZGJveEFkc0FQSXMiLCJleHBpcnkiOjE2OTUxNjc5OTksImlzU3ViZG9tYWluIjp0cnVlfQ==" http-equiv="origin-trial"/>
  <script nonce="cbGzRXMBKOar9uUBXAdjWg">
   var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else{var k;for(k in a[0])ytcfg.d()[k]=a[0][k]}}};

In [30]:
soup.find_all('h3')

[]

In [9]:
def build_metadata_dict(ytobject: YouTube):
    meta = {}
    items = ['channel_id', 'description', 'keywords', 'length', 'metadata', 'publish_date', 'thumbnail_url', 'title', 'views']
    meta['channel_id'] = ytobject.channel_id
    meta['description'] = ytobject.description if ytobject.description else "No description provided" 
    meta['keywords'] = ytobject.keywords
    meta['length'] = ytobject.length
    meta['metadata'] = ytobject.metadata
    meta['publish_date'] = ytobject.publish_date
    meta['thumbnail_url'] = ytobject.thumbnail_url
    title = ytobject.title.split('|')[0].strip() if '|' in ytobject.title else ytobject.title
    meta['title'] = title
    meta['views'] = ytobject.views
    return meta

In [10]:
def get_meta_download_audio(link: str, download_audio: bool=False, audio_dest: str='./'):
    try:
        video = YouTube(link)
        meta = build_metadata_dict(video)
        if download_audio:
            audio = video.streams.filter(only_audio=True, file_extension='mp4').first()
            audio.download(audio_dest, skip_existing=False)
        return meta
    except Exception as e:
        print(e)

In [11]:
test = [get_meta_download_audio(link) for link in links]

In [12]:
len(test)

5

In [13]:
thumbnails = [d['thumbnail_url'] for d in test]
thumbnails

['https://i.ytimg.com/vi/Qd5-OBopiCQ/hq720.jpg',
 'https://i.ytimg.com/vi/Kqnnvpfqxt0/hq720.jpg',
 'https://i.ytimg.com/vi/R6M9djUFJ60/hq720.jpg',
 'https://i.ytimg.com/vi/0H9CfT351U0/hq720.jpg',
 'https://i.ytimg.com/vi/ShEK9YEZ45Y/hq720.jpg']

In [14]:
test

[{'channel_id': 'UCXKjhxsfFQUqlNVQzLVnpEA',
  'description': 'No description provided',
  'keywords': [],
  'length': 2812,
  'metadata': <pytube.metadata.YouTubeMetadata at 0x7fd1f0578a60>,
  'publish_date': datetime.datetime(2023, 8, 24, 0, 0),
  'thumbnail_url': 'https://i.ytimg.com/vi/Qd5-OBopiCQ/hq720.jpg',
  'title': 'Lessons From A Trump-Less Debate',
  'views': 44793},
 {'channel_id': 'UCXKjhxsfFQUqlNVQzLVnpEA',
  'description': 'No description provided',
  'keywords': [],
  'length': 1715,
  'metadata': <pytube.metadata.YouTubeMetadata at 0x7fd1f04ff1c0>,
  'publish_date': datetime.datetime(2023, 8, 21, 0, 0),
  'thumbnail_url': 'https://i.ytimg.com/vi/Kqnnvpfqxt0/hq720.jpg',
  'title': 'What The GOP Primary Looks Like In The Early States',
  'views': 21349},
 {'channel_id': 'UCXKjhxsfFQUqlNVQzLVnpEA',
  'description': 'No description provided',
  'keywords': [],
  'length': 2310,
  'metadata': <pytube.metadata.YouTubeMetadata at 0x7fd1f0709e80>,
  'publish_date': datetime.dat

In [112]:
list(zip(thumbnails, titles))

[('https://i.ytimg.com/vi/Qd5-OBopiCQ/hq720.jpg',
  'Lessons From A Trump-Less Debate'),
 ('https://i.ytimg.com/vi/Kqnnvpfqxt0/hq720.jpg',
  'What The GOP Primary Looks Like In The Early States'),
 ('https://i.ytimg.com/vi/R6M9djUFJ60/hq720.jpg',
  "Three Georgia Law Professors Weigh In On Trump's Latest Indictment"),
 ('https://i.ytimg.com/vi/0H9CfT351U0/hq720.jpg',
  'Who Is The Likeliest GOP VP Candidate?'),
 ('https://i.ytimg.com/vi/ShEK9YEZ45Y/hq720.jpg',
  "The 2024 Election According To The Country's Best Pollster")]

In [52]:
with open('./data/impact_theory_with_vectors.json') as f:
    data = json.loads(f.read())

In [51]:
utils = Utilities()

In [55]:
data = utils.load_parquet('./data/impact_theory_with_vectors.parquet')

Shape of data: (33164, 18)
Memory Usage: 4.33+ MB


In [57]:
for item in data:
    assert len(item['content_embedding']) == 384

In [10]:
import googlesearch

In [18]:
results = googlesearch.search('llamaindex')

In [24]:
results = [i for i in results]
urls = list(set(results))

In [1]:
from llama_index import download_loader

SimpleWebPageReader = download_loader("SimpleWebPageReader")

loader = SimpleWebPageReader()
documents = loader.load_data(urls=['https://google.com'])

In [11]:
def get_docs_from_web(query: str):
    results = googlesearch.search('llamaindex')
    docs = loader.load_data(urls=[*results])
    return [d.text for d in docs]

In [22]:
res

['https://medium.com/llamaindex-blog/introducing-llamaindex-ts-89f41a1f24ab',
 'https://www.linkedin.com/company/llamaindex',
 'https://gpt-index.readthedocs.io/']

In [12]:
docs = get_docs_from_web('llamaindex')

In [30]:
from langchain.document_loaders import WebBaseLoader
import re

In [26]:
loader = WebBaseLoader(web_path=urls, verify_ssl=False)

In [27]:
docs = loader.load()

/anaconda/envs/llama/lib/python3.10/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'medium.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/anaconda/envs/llama/lib/python3.10/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.linkedin.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/anaconda/envs/llama/lib/python3.10/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gpt-index.readthedocs.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/anaconda/envs/llama/lib/python3

In [33]:
for doc in docs:
    re.sub('\n+', '\n', doc.page_content)

In [40]:
def remove_spaces(content: str):
    content = re.sub('\n+', '\n', content)
    content = re.sub('\s+', ' ', content)
    return content

In [43]:
len(remove_spaces(docs[1].page_content).split())

1561

Bad pipe message: %s [b'\x16\x00CO\x91\xac\xff\xb5\x19h\n\xe1|cR\xa3\x92T ]\xdf\n\xd6K!ji9\xdd\xee92\xbb\x0b\x12#\xcc\x9b7\xe1\x9a\x0cn\xfd)\xd94\xeby\xc1\x84\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00']
Bad pipe message: %s [b'\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05']
Bad pipe message: %s [b'\x03\x08']
Bad pipe message: %s [b'\x08\x08\t\x08\n\x08']
Bad pipe message: %s [b'Y\xc6\xfc=\xf7H\xa0\xf2#\x8a!T\xb7\xf1\xb5 x\xf8 z0\xcb\x196h\xb9\xde\xafU$V\xe1R8b#\xf3Ft\x11j@\x90|{\x1d\xd9D\x1a\x95\x01\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00']
Bad pipe message: %s [b'\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06']
Bad pipe message: %s [b'\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\x1d\x00\x17\x00\x1e\x00\x19\x00